In [ ]:
! pip install accelerate peft bitsandbytes pip install git+https://github.com/huggingface/transformers trl py7zr auto-gptq

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-fm0ea67t
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-fm0ea67t
  Resolved https://github.com/huggingface/transformers to commit c9f6e5e35156e068b227dd9b15521767f6afd4d2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
! pip install optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0.dev0
    Uninstalling transformers-4.40.0.dev0:
      Successfully uninstalled transformers-4.40.0.dev0


In [ ]:
!pip install auto-gptq

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, GPTQConfig, GenerationConfig
from trl import SFTTrainer
import os

In [ ]:
device = "cuda"
model_id = "ananyarn/new_python_convert"
model = AutoPeftModelForCausalLM.from_pretrained("ananyarn/new_python_convert", low_cpu_mem_usage=True, return_dict=True, torch_dtype=torch.float16, device_map="cuda").to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

CUDA extension not installed.
CUDA extension not installed.


In [ ]:
test = load_dataset("ananyarn/Algorithm_and_Python_Source_Code", split="train")[:50]
encodings = tokenizer("\n\n".join(test["Algorithm"]), return_tensors="pt")

In [ ]:
!pip install tqdm

In [ ]:
import torch
from tqdm import tqdm

max_length = 8192
stride = 512
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())

 45%|████████████████████▌                         | 13/29 [00:57<01:11,  4.45s/it]


In [ ]:
ppl

tensor(2.0343, device='cuda:0')

In [ ]:
print(ppl)

tensor(2.0343, device='cuda:0')
